In [5]:
#read the csv file
import pandas as pd
can = pd.read_csv("https://raw.githubusercontent.com/medcab9/ds/master/data/Cannabis%20StrainsFeatures%20-%20cannabis.csv")
print(can.shape)
can.head()

(2351, 6)


,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [2]:
can.describe(include='all')

,Strain,Type,Rating,Effects,Flavor,Description
count,2351,2351,2351.000000,2351,2350,2350
unique,2350,3,NaN,1704,1349,2343
top,B-Witched,hybrid,NaN,None,None,None
freq,2,1212,NaN,29,44,5
mean,NaN,NaN,4.382561,NaN,NaN,NaN
std,NaN,NaN,0.639090,NaN,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,NaN
25%,NaN,NaN,4.200000,NaN,NaN,NaN
50%,NaN,NaN,4.500000,NaN,NaN,NaN
75%,NaN,NaN,4.700000,NaN,NaN,NaN


In [3]:
can.isnull().sum()

Strain         0
Type           0
Rating         0
Effects        0
Flavor         1
Description    1
dtype: int64

In [6]:
# Cleaning the data and changing nan values to unknowns and 0s
import numpy as np

can['Rating'] = can['Rating'].replace(np.nan, 0)
can['Effects'] = can['Effects'].replace(np.nan, 'Effects currently unknown')
can['Flavor'] = can['Flavor'].replace(np.nan, 'Flavor profile currently unavailable')
can['Description'] = can['Description'].replace(np.nan, 'Strain description currently unavailable')
can['Type'] = can['Type'].replace(np.nan, 'Type currently unknown')


In [5]:
can.isnull().sum()

Strain         0
Type           0
Rating         0
Effects        0
Flavor         0
Description    0
dtype: int64

In [6]:
# Unique Strain types

can.Strain.value_counts()

B-Witched            2
Woody-Kush           1
Magnum-Pi            1
Funfetti             1
Throwback-Kush       1
                    ..
Lennon               1
Lvpk                 1
Blue-Rhino           1
Electric-Lemonade    1
Aceh                 1
Name: Strain, Length: 2350, dtype: int64

In [7]:
can['Rating'].unique()

array([4. , 4.7, 4.4, 4.2, 4.6, 4.5, 4.3, 5. , 3.8, 4.1, 4.8, 3.4, 3.7,
       3.9, 0. , 3. , 4.9, 3.6, 2.8, 3.3, 3.5, 3.2, 1. , 3.1, 2.5])

In [8]:
# Create new series with information

can['total_text'] = can.Strain + can.Effects + can.Description

In [9]:
can.head()

,Strain,Type,Rating,Effects,Flavor,Description,total_text
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,"100-OgCreative,Energetic,Tingly,Euphoric,Relax..."
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,"98-White-WidowRelaxed,Aroused,Creative,Happy,E..."
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,"1024Uplifted,Happy,Relaxed,Energetic,Creative1..."
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,"13-DawgsTingly,Creative,Hungry,Relaxed,Uplifte..."
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...","24K-GoldHappy,Relaxed,Euphoric,Uplifted,Talkat..."


In [11]:
# df = pd.read_csv("nn_model_strains.csv")
# df.head()

,Unnamed: 0,Strain,Type,Effects,Flavor,Description,name,ailment,all_text,lemmas
0,0,100-Og,hybrid,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,NaN,NaN,"100-Og hybrid Creative,Energetic,Tingly,Euphor...",100-Og hybrid Creative energetic Tingly Euphor...
1,1,98-White-Widow,hybrid,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,NaN,NaN,"98-White-Widow hybrid Relaxed,Aroused,Creative...",98-white widow hybrid Relaxed arouse Creative ...
2,2,1024,sativa,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,1024,"Stress, Pain, Depression, Inflammation","1024 sativa Uplifted,Happy,Relaxed,Energetic,C...",1024 sativa Uplifted happy relaxed energetic C...
3,3,13-Dawgs,hybrid,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,NaN,NaN,"13-Dawgs hybrid Tingly,Creative,Hungry,Relaxed...",13-dawgs hybrid Tingly creative hungry Relaxed...
4,4,24K-Gold,hybrid,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...",NaN,NaN,"24K-Gold hybrid Happy,Relaxed,Euphoric,Uplifte...",24k gold hybrid Happy Relaxed Euphoric Uplifte...


In [12]:
# df.shape

(2280, 10)

## 2) Use Spacy to tokenize the total text

In [2]:
import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
from spacy.tokenizer import Tokenizer
from bs4 import BeautifulSoup

In [3]:
# Inialize spaCy model and tokenizer
nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)

In [10]:
tokens = []

# Return lower case tokens and ignore stop words/punctuation
for doc in tokenizer.pipe(can['total_text'], batch_size=500):
    
    doc_tokens = []
    
    for token in doc:
        # ignore stopwords, spaces and punctuation
        if (token.is_stop == False) & (token.is_punct == False) & (token.is_space == False):
            doc_tokens.append(token.text.lower())

    tokens.append(doc_tokens)

can['spaCy_tokens'] = tokens
can['spaCy_tokens'].head()

0    [100-ogcreative,energetic,tingly,euphoric,rela...
1    [98-white-widowrelaxed,aroused,creative,happy,...
2    [1024uplifted,happy,relaxed,energetic,creative...
3    [13-dawgstingly,creative,hungry,relaxed,uplift...
4    [24k-goldhappy,relaxed,euphoric,uplifted,talka...
Name: spaCy_tokens, dtype: object

In [11]:
print(can.shape)
can.head()

(2351, 8)


,Strain,Type,Rating,Effects,Flavor,Description,total_text,spaCy_tokens
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,"100-OgCreative,Energetic,Tingly,Euphoric,Relax...","[100-ogcreative,energetic,tingly,euphoric,rela..."
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,"98-White-WidowRelaxed,Aroused,Creative,Happy,E...","[98-white-widowrelaxed,aroused,creative,happy,..."
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,"1024Uplifted,Happy,Relaxed,Energetic,Creative1...","[1024uplifted,happy,relaxed,energetic,creative..."
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,"13-DawgsTingly,Creative,Hungry,Relaxed,Uplifte...","[13-dawgstingly,creative,hungry,relaxed,uplift..."
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...","24K-GoldHappy,Relaxed,Euphoric,Uplifted,Talkat...","[24k-goldhappy,relaxed,euphoric,uplifted,talka..."
